<a href="https://colab.research.google.com/github/szh141/Examples/blob/main/Patient_data_preprocessing_for_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://levelup.gitconnected.com/mastering-feature-engineering-process-in-data-science-6897ba5a2d7a



#Conclusion

Throughout our analysis, we started with the goal of optimizing a dataset for a machine learning model focused on predicting the variable “Developed_Pneumonia”. We began by identifying and removing redundant or irrelevant variables from the dataset, ensuring a more streamlined and effective data base.

Next, we implemented encoding techniques to transform categorical variables into numerical formats, essential for predictive analysis.

The crucial step was the selection of relevant features. We used the RandomForestClassifier, a powerful tool for assessing variable importance. With this, we identified the most significant features for predicting pneumonia, allowing us to focus on data that truly influences the outcome. The selection process was optimized and streamlined for efficiency, resulting in clear and straightforward code.

Finally, we created an optimized dataset, containing only the most relevant features and the target variable, and saved it in CSV format.

This refined dataset is now ready to be applied in predictive models, potentially improving accuracy and performance. In summary, our work here established a solid foundation for building an efficient and reliable machine learning model for pneumonia prediction.

In [ ]:
import pandas as pd

# Load the dataset
file_path = 'dados_pacientes.csv'
dataset = pd.read_csv(file_path)

# Display the first few rows of the dataset to understand its structure
dataset.head()


In [ ]:
# Calculating statistical summary for numerical variables
numerical_summary = dataset.describe()

# Calculating summary for categorical variables
categorical_summary = dataset.describe(include=['object'])

numerical_summary, categorical_summary


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Calculando a correlação entre Peso e Altura
correlation = dataset['Peso'].corr(dataset['Altura'])

# Criando um scatter plot para visualizar a relação entre Peso e Altura
plt.figure(figsize=(10, 6))
sns.scatterplot(data=dataset, x='Altura', y='Peso')
plt.title('Relazione tra Peso e Altezza dei Pazienti')
plt.xlabel('Altezza (cm)')
plt.ylabel('Peso (kg)')
plt.grid(True)

# Mostrando o coeficiente de correlação
plt.figtext(0.5, 0.01, f"Coefficiente di Correlazione: {correlation:.2f}", ha="center", fontsize=12, bbox={"facecolor":"orange", "alpha":0.5, "pad":5})

plt.show()

correlation

# 0.365

In [ ]:
# Calcolo dell'Indice di Massa Corporea (IMC)
dataset['IMC'] = (dataset['Peso'] / (dataset['Altura'] / 100) ** 2).round(0)

# Visualizzando le prime righe del dataset per verificare la nuova colonna
dataset.head()


In [ ]:
try:
    # Calculando o IMC
    dataset['IMC'] = (dataset['Peso'] / ((dataset['Altura'] / 100) ** 2)).round(0)

    # Criando o gráfico de distribuição do IMC
    plt.figure(figsize=(10, 6))
    sns.histplot(dataset['IMC'], bins=20, kde=True)
    plt.title('Distribuzione dell\'Indice di Massa Corporea (IMC) dei Pazienti')
    plt.xlabel('IMC')
    plt.ylabel('Frequenza')
    plt.grid(True)
    plt.show()
except Exception as e:
    print(f"Ocorreu um erro: {e}")


In [ ]:
# Removendo as variáveis Peso e Altura do dataset
dataset = dataset.drop(['Peso', 'Altura'], axis=1)

# Exibindo as primeiras linhas do dataset para verificar as alterações
dataset.head()


In [ ]:
# Criando uma nova coluna 'Codigo_Hist_Paciente' com os primeiros 5 dígitos da coluna 'ID_Paciente'
dataset['Codigo_Hist_Paciente'] = dataset['ID_Paciente'].str[:5]

# Exibindo as primeiras linhas do dataset para verificar a nova coluna
dataset.head()


In [ ]:
# Verificando o total de valores únicos na coluna 'Codigo_Hist_Paciente'
try:
    unique_codes = dataset['Codigo_Hist_Paciente'].nunique()
    unique_codes
except Exception as e:
    unique_codes = "Erro ao calcular: " + str(e)

unique_codes


In [ ]:
# Criando uma nova coluna 'faixa_etaria' com base na coluna 'Idade'
# Definindo as faixas etárias
bins = [0, 20, 30, 45, 50, 100]
labels = ['0-20', '20-30', '30-40', '40-50', '50+']

# Usando a função cut do pandas para criar as categorias
dataset['faixa_etaria'] = pd.cut(dataset['Idade'], bins=bins, labels=labels, right=False)

# Exibindo as primeiras linhas do dataset para verificar a nova coluna
dataset.head()


In [ ]:
# Relação entre ser fumante e o desenvolvimento de pneumonia
relation_smoking_pneumonia = dataset[['Fumante', 'Desenvolveu_Pneumonia']].groupby(['Fumante']).agg('count')
relation_smoking_pneumonia

In [ ]:
from matplotlib import pyplot as plt
relation_smoking_pneumonia['Desenvolveu_Pneumonia'].plot(kind='hist', bins=20, title='Desenvolveu_Pneumonia')
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
import pandas as pd

# Criando uma tabela de contingência entre as variáveis 'Fumante' e 'Desenvolveu_Pneumonia'
contingency_table = pd.crosstab(dataset['Fumante'], dataset['Desenvolveu_Pneumonia'])

contingency_table


In [ ]:
from scipy.stats import chi2_contingency
# Creando la tabella di contingenza per i dati di fumatori e sviluppo della polmonite
tabella_contingenza = [[278, 268], [220, 234]]

# Esecuzione del test del Chi-quadrato
chi2, p, dof, expected = chi2_contingency(tabella_contingenza)
chi2, p

In [ ]:
# Definindo as faixas
bins = [-1, 2, 4, dataset['Numero_Filhos'].max()]

# Definindo os rotulos para as faixas
labels = ['0-2', '3-4', '5+']

# Criando a nova coluna categórica
dataset['Faixa_Filhos'] = pd.cut(dataset['Numero_Filhos'], bins=bins, labels=labels, right=True)

# Exibindo as primeiras linhas do dataset para verificar a nova coluna
dataset.head()

In [ ]:
# Colunas a serem removidas
colunas_para_remover = ['ID_Paciente', 'Data_Nascimento', 'Idade', 'Numero_Filhos', 'Codigo_Hist_Paciente']

# Removendo as colunas
clean_dataset = dataset.drop(columns=colunas_para_remover)
clean_dataset.head()

In [ ]:
# Lista para armazenar os resultados
results = []

# Colunas categoricas
categorical_columns = clean_dataset.select_dtypes(include=[pd.np.object]).columns.tolist()
categorical_columns.remove('Desenvolveu_Pneumonia')

# Loop por todas as colunas categoricas
for col in categorical_columns:

# Criando a tabela de contingencia
  contingency_table = pd.crosstab(clean_dataset[col], clean_dataset['Desenvolveu_Pneumonia'])

  # Realizando o teste do qui-quadrado
  chi2, p_value, dof, expected = chi2_contingency(contingency_table)

  # Adicionando os resultados a lista
  results.append({'Variable': col, 'p-value': p_value})

# Convertendo os resultados em um DataFrame para uma visualizacao mais facil
results_df = pd.DataFrame(results)
results_df

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Criando os codificadores
le = LabelEncoder()
ohe = OneHotEncoder(drop = 'first', sparse = False)

# Codificando a variavel alvo
clean_dataset['Desenvolveu_Pneumonia'] = le.fit_transform(clean_dataset['Desenvolveu_Pneumonia'])

# Lista para armazenar as novas colunas categoricas co
new_categorical_cols = []

# Loop por todas as colunas categoricas
for col in categorical_columns:

  # Codificando a coluna
  encoded_cols = ohe.fit_transform(clean_dataset[[col]])

  # Transformando o resultado em um DataFrame e Adicionar
  encoded_cols_df = pd.DataFrame(encoded_cols, columns = [f"{col}_{category}" for category in ohe.categories_[0][1:]])
  # Adicionando o DataFrame resultante a lista
  new_categorical_cols.append(encoded_cols_df)

# Concatenando todos os DataFrames da lista
new_categorical_cols_clean_dataset= pd.concat(new_categorical_cols, axis=1)

# Removendo as colunas categoricas originais do DataFrame
cleand_dataset = clean_dataset.drop(categorical_columns, axis=1)

# Adicionando as novas colunas categoricas codificada
df = pd.concat([clean_dataset, new_categorical_cols_clean_dataset], axis=1)

# Primeiras linhas
df.head()


In [ ]:

# Aplicando One-Hot Encoding para 'faixa_etaria' e 'Faixa_Filhos' usando pd.get_dummies()
df = pd.get_dummies(df, columns=['faixa_etaria', 'Faixa_Filhos'], drop_first=True)

# Visualizando as primeiras linhas do DataFrame atualizado
print(df.head())


In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import OneHotEncoder

# Carregar o DataFrame
# df = pd.read_csv('seu_dataset.csv')

# Identificar colunas categóricas
categorical_cols = df.select_dtypes(include=['object', 'category']).columns

# Aplicar One-Hot Encoding para variáveis categóricas
df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

# Separar características e variável alvo
X = df_encoded.drop('Desenvolveu_Pneumonia', axis=1)
y = df_encoded['Desenvolveu_Pneumonia']

# Criar e treinar o modelo Random Forest
model = RandomForestClassifier()
model.fit(X, y)

# Selecionar características com base na importância
selector = SelectFromModel(model, threshold='median')
X_selected = selector.fit_transform(X, y)

# Obter os nomes das características selecionadas
selected_features = X.columns[selector.get_support()]

print("Características selecionadas:", selected_features)


In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

# Aplicar One-Hot Encoding e separar as características e a variável alvo
df_encoded = pd.get_dummies(df, drop_first=True)
X = df_encoded.drop('Desenvolveu_Pneumonia', axis=1)
y = df['Desenvolveu_Pneumonia']

# Treinar o modelo Random Forest e selecionar características importantes
model = RandomForestClassifier().fit(X, y)
selected_features = X.columns[SelectFromModel(model, threshold='median').fit(X, y).get_support()]

# Criar um novo DataFrame com as características selecionadas e a variável alvo
df_final = df_encoded[selected_features].join(df['Desenvolveu_Pneumonia'])

# Salvar o novo DataFrame em um arquivo CSV
df_final.to_csv('dataset_otimizado.csv', index=False)

print("Dataset otimizado salvo como 'dataset_otimizado.csv'.")
